## Preparations

In [1]:
# !auto-py-to-exe

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
print("d2loadout update started")

d2loadout update started


In [4]:
import json
import os
import re
import pandas as pd
import numpy as np
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import ast
import html_to_json

In [5]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("user-agent=" + UserAgent().random)
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("detach", True)

# chrome_options.add_argument('headless')
# chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument("disable-gpu")

chrome_options.page_load_strategy = 'eager'


os.environ["PATH"] += r"C:\Program Files (x86)\Chromedriver.exe"
driver = webdriver.Chrome(options=chrome_options)

## Meta

In [6]:
link_meta = "https://dota2protracker.com/meta"

In [7]:
driver.implicitly_wait(3)
driver.get(link_meta)
driver.implicitly_wait(3)

In [8]:
time.sleep(1)
category_name_elements = driver.find_elements(By.XPATH,"//a[@href]")
all_hrefs = [elem.get_attribute("href") for elem in category_name_elements]
links = [link for link in all_hrefs if "/hero/" in link]
links = sorted(list(set(links)))

In [9]:
# link = links[links.index("https://dota2protracker.com/hero/Shadow%20Fiend")]
# driver.get(link)
# s = "flex w-[50px] gap-1 flex-col bg-d2pt-gray-1 p-1 rounded-md border-one text-xs text-white font-medium"
# "." + ".".join([sub for sub in s.split(" ") if ":" not in sub and "[" not in sub])

In [10]:
def get_build(driver):
    rows = driver.find_element(By.CSS_SELECTOR, '.grid.grid-cols-3.p-4.gap-1.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8')\
    .find_elements(By.XPATH, "./child::*")
    rows
    talents = rows[0]\
        .find_elements(By.CSS_SELECTOR, ".abs.svelte-yv4bey")
    talents_classtext = [talent.get_attribute("class") for talent in talents]
    build_talents = [re.findall("(\d+)\s(rt|lt)" , talents_res) for talents_res in talents_classtext]
    build_talents = [t[0] for t in build_talents if t]
    build_talents = [int(x[1] == "rt") for x in sorted(build_talents)]
    build_talents
    abilities = rows[0]\
        .find_element(By.CSS_SELECTOR, '.flex.flex-wrap.gap-1.items-center').find_elements(By.XPATH, "./child::img")
    build_abilities = [a.get_attribute("title") for a in abilities]
    build_abilities
    starting = rows[1]\
        .find_elements(By.CSS_SELECTOR, '.flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one')
    starting = [div.find_elements(By.XPATH, "./child::div")[0] for div in starting]
    build_starting = [a.get_attribute("title") for a in starting]
    build_starting
    common_late = rows[2]\
        .find_elements(By.CSS_SELECTOR, '.text-xs.text-white.font-medium.text-right.rounded-md.text-shadow')
    build_common_late = [a.get_attribute("title") for a in common_late]
    build_common_late
    rows_2 = driver.find_elements(By.CSS_SELECTOR, '.grid.grid-cols-3.gap-1.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8')[1]\
    .find_elements(By.XPATH, "./child::*")
    rows_2
    early = rows_2[0]\
        .find_elements(By.CSS_SELECTOR, '.flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one')
    early = [div.find_elements(By.XPATH, "./child::div")[0] for div in early]
    build_early = [a.get_attribute("title") for a in early]
    build_early
    mid = rows_2[1]\
        .find_elements(By.CSS_SELECTOR, '.flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one')
    mid = [div.find_elements(By.XPATH, "./child::div")[0] for div in mid]
    build_mid = [a.get_attribute("title") for a in mid]
    build_mid
    late = rows_2[2]\
        .find_elements(By.CSS_SELECTOR, '.flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one')
    late = [div.find_elements(By.XPATH, "./child::div")[0] for div in late]
    build_late = [a.get_attribute("title") for a in late]
    build_late
    core = driver.find_element(By.CSS_SELECTOR, '.flex.mb-2.flex-col.p-4.gap-2.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8.overflow-x-auto')\
    .find_elements(By.CSS_SELECTOR, '.flex.gap-1.flex-col.bg-d2pt-gray-1.p-1.rounded-md.border-one.text-xs.text-white.font-medium')
    build_core = [c.find_elements(By.XPATH, "./child::div")[1].get_attribute("title") for c in core]
    build_core
    return [build_talents, build_abilities, build_starting, build_common_late, build_early, build_mid, build_late, build_core]

In [11]:
# arr_df = []
# for link in links:
#     driver.get(link)
#     driver.implicitly_wait(0.7)
#     hero_name = link.split("/")[-1].replace("%20", " ")
#     hero_name
#     positions = driver.find_element(By.CSS_SELECTOR, ".flex.flex-col.gap-2.sticky.top-1").find_elements(By.XPATH, "./child::*")[1:]

#     for position in positions:
#         position.click()
#         position_text = position.text.split("\n")[0]
#         facets = driver.find_elements(By.CSS_SELECTOR, '.cursor-pointer.flex.flex-col.rounded-md.relative')
#         for facet in facets:
#             facet.click()
#             facet_text = facet.text.split("\n")[-1]
#             builds = driver.find_elements(By.CSS_SELECTOR, '.rounded-md.svelte-15r7zt1')
#             for build in builds:
#                 build.click()
#                 build_matches = re.findall("\d+ matches", build.text)[0].split(" ")[0]
#                 build_win_rate = re.findall("\d+% win rate", build.text)[0].split("%")[0]
#                 arr_df.append([hero_name, position_text, facet_text, build_matches, build_win_rate] + get_build(driver))


In [12]:
# cols_arr = ["hero_name", "position","facet","matches", "build_win_rate", "talents", "abilities", "starting", "common_late", "early", "mid", "late", "core"]
# cols_dict = dict(zip(range(len(cols_arr)), cols_arr))
# temp = pd.DataFrame(dict(zip(range(len(arr_df)), arr_df))).T
# temp = temp.rename(cols_dict,axis=1)
# temp

In [13]:
# temp.to_csv("./parsed_data.csv")

In [14]:
temp = pd.read_csv("./parsed_data.csv")
temp

,Unnamed: 0,hero_name,position,facet,matches,build_win_rate,talents,abilities,starting,common_late,early,mid,late,core
0,0,Abaddon,Carry,MEPHITIC SHROUD,31,48,"[1, 1, 0]","['Aphotic Shield', 'Curse of Avernus', 'Aphoti...","['Tango', 'Quelling Blade', 'Iron Branch', 'Ci...","['Phase Boots', 'Radiance', 'Manta Style', 'Ha...","['Bracer', 'Wraith Band', 'Magic Wand', 'Null ...","['Radiance', 'Blade Mail', 'Yasha', ""Aghanim's...","[""Linken's Sphere"", 'Eye of Skadi', ""Aghanim's...","['Bracer', 'Magic Wand', 'Boots of Speed', 'Ph..."
1,1,Abaddon,Offlane,THE QUICKENING,32,44,"[1, 1, 0]","['Aphotic Shield', 'Curse of Avernus', 'Aphoti...","['Tango', 'Circlet', 'Circlet', 'Quelling Blad...","['Phase Boots', ""Aghanim's Scepter"", 'Radiance...","['Wraith Band', 'Bracer', 'Boots of Speed', 'N...","['Blade Mail', 'Armlet of Mordiggian', 'Echo S...","['Skull Basher', 'Abyssal Blade', 'Divine Rapi...","['Bracer', 'Boots of Speed', 'Orb of Corrosion..."
2,2,Abaddon,Offlane,MEPHITIC SHROUD,39,49,"[1, 1, 0]","['Aphotic Shield', 'Curse of Avernus', 'Aphoti...","['Iron Branch', 'Tango', 'Magic Wand', 'Circle...","['Phase Boots', 'Radiance', 'Harpoon', 'Scythe...","['Magic Wand', 'Wraith Band', 'Bracer', 'Boots...","['Blade Mail', 'Mage Slayer', 'Diffusal Blade'...","['Disperser', 'Scythe of Vyse', 'Nullifier', '...","['Magic Wand', 'Bracer', 'Harpoon', 'Echo Sabr..."
3,3,Abaddon,Support,THE QUICKENING,27,70,"[0, 0, 1]","['Aphotic Shield', 'Mist Coil', 'Mist Coil', '...","['Tango', 'Blood Grenade', 'Sentry Ward', 'Mag...","[""Aghanim's Scepter"", 'Holy Locket', 'Arcane B...","['Boots of Speed', 'Wind Lace', 'Magic Wand', ...","['Pavise', 'Holy Locket', 'Drum of Endurance',...","['Pipe of Insight', 'Octarine Core', 'Blink Da...","['Boots of Speed', 'Wind Lace', 'Magic Wand', ..."
4,4,Abaddon,Support,THE QUICKENING,27,48,[0],"['Aphotic Shield', 'Mist Coil', 'Mist Coil', '...","['Tango', 'Blood Grenade', 'Magic Stick', 'Sen...",[],"['Ring of Basilius', 'Boots of Speed', 'Magic ...","['Mekansm', 'Holy Locket', 'Pavise', 'Solar Cr...",[],"['Ring of Basilius', 'Boots of Speed', 'Magic ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,835,Zeus,Support,LIVEWIRE,530,57,"[0, 1, 0, 1]","['Arc Lightning', 'Heavenly Jump', 'Arc Lightn...","['Tango', 'Blood Grenade', 'Sentry Ward', 'Enc...","[""Aghanim's Scepter"", 'Arcane Boots', 'Refresh...","['Ring of Basilius', 'Boots of Speed', 'Null T...","['Wind Lace', 'Phylactery', 'Aether Lens', ""Ag...","['Yasha and Kaya', 'Octarine Core', 'Refresher...","['Ring of Basilius', 'Boots of Speed', 'Null T..."
836,836,Zeus,Support,LIVEWIRE,142,33,"[0, 1, 1]","['Arc Lightning', 'Heavenly Jump', 'Arc Lightn...","['Tango', 'Blood Grenade', 'Observer and Sentr...",[],"['Ring of Basilius', 'Boots of Speed', 'Magic ...","['Phylactery', ""Aghanim's Shard"", 'Glimmer Cap...",[],"['Ring of Basilius', 'Boots of Speed', 'Null T..."
837,837,Zeus,Support,DIVINE RAMPAGE,292,53,"[0, 1, 0, 0]","['Lightning Bolt', 'Heavenly Jump', 'Lightning...","['Tango', 'Blood Grenade', 'Enchanted Mango', ...","['Arcane Boots', ""Aghanim's Scepter"", 'Refresh...","['Ring of Basilius', 'Null Talisman', 'Boots o...","['Wind Lace', 'Phylactery', 'Aether Lens', 'Fo...","['Refresher Orb', 'Scythe of Vyse', 'Kaya and ...","['Ring of Basilius', 'Null Talisman', 'Boots o..."
838,838,Zeus,Hard Support,LIVEWIRE,322,50,"[0, 1, 0, 1]","['Lightning Bolt', 'Heavenly Jump', 'Lightning...","['Tango', 'Blood Grenade', 'Sentry Ward', 'Enc...","['Arcane Boots', ""Aghanim's Scepter"", 'Refresh...","['Ring of Basilius', 'Boots of Speed', 'Null T...","['Wind Lace', 'Phylactery', 'Force Staff', 'Gl...","['Ethereal Blade', 'Octarine Core', 'Boots of ...","['Ring of Basilius', 'Boots of Speed', 'Null T..."


In [15]:
driver.implicitly_wait(3)

In [16]:
def get_d2pt_page_table(driver):
    # finding table rows in site and converting to format, readable by pandas
    time.sleep(0.2)
    category_name_elements = driver.find_elements(By.XPATH, "//*[@data-wr]")
    div_inner_strs = []
    for elem in category_name_elements:
        div_inner_str = (
            elem.get_attribute("outerHTML").split(">")[0].replace("data-", "")
        )
        div_items = re.findall(
            r'[-\w]+="[\w\d\s.,\/#!$%\^&\*;:{}=\-_`~()\'\\/\[\]]+"', div_inner_str
        )
        temp_dict = {}
        for item in div_items:
            temp_split = item.split("=")
            temp_dict[temp_split[0]] = temp_split[1].replace('"', "")
        div_inner_strs.append(temp_dict)

    # Dataframe dtype converts
    df_heroes_table = pd.DataFrame(data=div_inner_strs)
    df_heroes_table = df_heroes_table.apply(pd.to_numeric, errors="ignore")
    df_heroes_table = df_heroes_table.round(1)
    return df_heroes_table

In [17]:
print("checking meta...")
print("fetching heroes info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table(driver)

print("fetching heroes info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table(driver)

df_full = pd.concat([df_1, df_2, df_3, df_4, df_5], axis=0)
df_full = df_full.drop(["class"], axis=1)
df_full

checking meta...
fetching heroes info for pos 1
fetching heroes info for pos 2
fetching heroes info for pos 3
fetching heroes info for pos 4
fetching heroes info for pos 5


,wr,pos,matches,contest,hero,rating,networth,phase1,phase2,phase3,expert,expertmatches,radiant,dire,elo,wr9500
0,56.5,pos 1,2546,60.1,Windranger,3708,4614,57.4,56.1,56.7,61.1,529,57.8,55.1,3708,58.2
1,53.9,pos 1,1260,55.8,Broodmother,3261,4643,52.3,54.9,51.9,58.0,312,55.6,52.1,3261,55.4
2,52.5,pos 1,5303,63.0,Ursa,3252,4433,49.0,51.1,56.1,57.5,856,54.7,50.2,3252,50.6
3,56.0,pos 1,877,11.3,Clinkz,3239,4383,53.1,56.1,56.4,62.6,182,57.6,54.4,3239,58.1
4,54.6,pos 1,434,19.2,Marci,3216,4714,60.4,54.4,48.3,73.0,63,56.1,52.9,3216,60.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,46.2,pos 5,13,22.2,Pangolier,0,2412,60.0,20.0,66.7,0.0,0,50.0,44.4,0,0.0
113,20.0,pos 5,10,11.8,Void Spirit,0,1843,0.0,50.0,0.0,0.0,0,33.3,0.0,0,22.2
114,28.6,pos 5,7,16.0,Mars,0,1979,25.0,50.0,0.0,0.0,0,50.0,0.0,0,0.0
115,40.0,pos 5,5,16.4,Primal Beast,0,2314,33.3,50.0,0.0,0.0,0,33.3,50.0,0,0.0


## Facets

In [18]:
link = "https://dota2protracker.com/facets"

In [19]:
driver.implicitly_wait(2)
driver.get(link)

In [20]:
hero_req = requests.get("https://dota2protracker.com/hero/Tiny")

In [21]:
facetData = re.findall(r'facetData:{.+}', hero_req.text)[0].replace("facetData:", "")

In [26]:
hero_req.text[:100]

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <meta name="viewport" content="wi'

In [22]:
import js2py

In [23]:
tojs = re.findall(r'const data = \[.+\"data\":{', hero_req.text)[0]
tojs = tojs + re.findall(r'facetData:{.+},buildData', hero_req.text)[0].replace(",buildData", "}}]")

IndexError: list index out of range

In [ ]:
js_res = js2py.eval_js(tojs) 

In [ ]:
d = js_res[2]["data"]["facetData"]
[d[item]["facets"] for item in d]
facets_d = {}
for item in d:
    facet_names = []
    for facets in d[item]["facets"]:
        facet_names.append(facets["name"].upper())
    facets_d[item] = facet_names
facets_d

{'1': ["MAGEBANE'S MIRROR", 'MANA THIRST'],
 '10': ['EBB', 'FLOW'],
 '100': ['TAG TEAM', 'DRINKING BUDDIES'],
 '101': ['SHIELD OF THE SCION', 'STAFF OF THE SCION'],
 '102': ['THE QUICKENING', 'MEPHITIC SHROUD'],
 '103': ['DECONSTRUCTION', 'MOMENTUM'],
 '104': ['STONEHALL PLATE', 'SPOILS OF WAR'],
 '105': ["SQUEE'S SCOPE", "SPLEEN'S SECRET SAUCE", "SPOON'S STASH"],
 '106': ['DOUBLE IMPACT', 'CHAIN GANG'],
 '107': ['RESONANCE', 'STEPPING STONE'],
 '108': ["DEMON'S REACH", 'ABYSSAL HORDE'],
 '109': ['CONDEMNED', 'SOUL FRAGMENT'],
 '11': ['LASTING PRESENCE', 'SHADOWMIRE'],
 '110': ['DYING LIGHT', 'HOTSPOT'],
 '111': ['CLAIRVOYANT CURSE', 'CLAIRVOYANT CURE'],
 '112': ['ESSENCE OF THE BLUEHEART', 'DRAGON SIGHT'],
 '113': ['ORDER', 'DISORDER'],
 '114': ["WUKONG'S FAITHFUL", 'SIMIAN STRIDE'],
 '119': ['THROWING SHADE', 'THORNY THICKET'],
 '12': ['CONVERGENCE', 'DIVERGENCE'],
 '120': ['DOUBLE JUMP', 'THUNDERBOLT'],
 '121': ['INKSTIGATE', 'FINE ART'],
 '123': ['GO NUTS', 'TREEBOUNCE TRICKSHOT'],

In [ ]:
# with open("./facetData.json", 'w+', encoding="utf-8") as f:
#     f.write(re.findall(r'facetData:{.+}', hero_req.text)[0].replace("facetData:", ""))

In [ ]:
def get_d2pt_page_table_facets(driver):
    time.sleep(0.2)
    # finding table rows in site and converting to format, readable by pandas
    category_name_elements = driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-3.gap-1"
    )
    category_name_elements += driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-4.gap-1"
    )
    hero_rows = [item.text.split("\n") for item in category_name_elements]
    hero_columns = [item for item in driver.find_element(
        By.CSS_SELECTOR, ".flex.gap-1.font-medium.text-sm.mb-1"
    ).text.split("\n") if item != "Trend"]
        
    # Dataframe dtype converts
    df_heroes_table = pd.DataFrame(data=hero_rows, columns=hero_columns)
    df_heroes_table = df_heroes_table.apply(pd.to_numeric, errors='ignore')
    df_heroes_table = df_heroes_table.round(1)
    return df_heroes_table

In [ ]:
print("checking facets...")
print("fetching facets info for pos 1")
driver.find_element(By.XPATH, "//div[contains(text(), 'Carry')]").click()
df_1 = get_d2pt_page_table_facets(driver)
df_1["pos"] = "pos 1"

print("fetching facets info for pos 2")
driver.find_element(By.XPATH, "//div[contains(text(), 'Mid')]").click()
df_2 = get_d2pt_page_table_facets(driver)
df_2["pos"] = "pos 2"

print("fetching facets info for pos 3")
driver.find_element(By.XPATH, "//div[contains(text(), 'Off')]").click()
df_3 = get_d2pt_page_table_facets(driver)
df_3["pos"] = "pos 3"

print("fetching facets info for pos 4")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 4')]").click()
df_4 = get_d2pt_page_table_facets(driver)
df_4["pos"] = "pos 4"

print("fetching facets info for pos 5")
driver.find_element(By.XPATH, "//div[contains(text(), 'Pos 5')]").click()
df_5 = get_d2pt_page_table_facets(driver)
df_5["pos"] = "pos 5"

df_full_facets = pd.concat([df_1,df_2,df_3,df_4,df_5], axis=0)

df_full_facets = df_full_facets.rename({
    "Hero": "hero",
    "Facet": "facet",
    "Matches": "matches",
    "Win Rate": "wr"
}, axis=1)[["hero", "facet", "matches", "wr", "pos"]]

df_full_facets["wr"] = df_full_facets["wr"].apply(lambda x: x.replace("%", ""))
df_full_facets = df_full_facets.apply(pd.to_numeric, errors='ignore')

df_full_facets

checking facets...
fetching facets info for pos 1
fetching facets info for pos 2
fetching facets info for pos 3
fetching facets info for pos 4
fetching facets info for pos 5


,hero,facet,matches,wr,pos
0,Anti-Mage,MAGEBANE'S MIRROR,204,41.2,pos 1
1,Bloodseeker,ARTERIAL SPRAY,96,44.8,pos 1
2,Drow Ranger,VANTAGE POINT,177,45.8,pos 1
3,Juggernaut,BLADEFORM,273,48.7,pos 1
4,Shadow Fiend,LASTING PRESENCE,155,51.6,pos 1
...,...,...,...,...,...
85,Dark Willow,THROWING SHADE,123,43.9,pos 5
86,Grimstroke,INKSTIGATE,325,47.7,pos 5
87,Snapfire,RICOCHET II,916,47.5,pos 5
88,Marci,SIDEKICK,742,45.7,pos 5


## Results

In [ ]:
print("creating in-game loadouts")

creating in-game loadouts


In [ ]:
req = requests.get("https://dota2protracker.com/_get/search").json()
hero_names = [item["displayName"] for item in req["heroes"]]
hero_ids = [item["hero_id"] for item in req["heroes"]]
hero_id = dict(zip(hero_names, hero_ids))
df_full["hero_id"] = df_full["hero"].apply(lambda x: hero_id[x])
df_full_facets["hero_id"] = df_full_facets["hero"].apply(lambda x: hero_id[x])

In [ ]:
df_full_facets["facet_id"] = df_full_facets.apply(lambda x: facets_d[f"{x.hero_id}"].index(x.facet), axis=1)

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
df_full_facets

,hero,facet,matches,wr,pos,hero_id,facet_id
0,Anti-Mage,MAGEBANE'S MIRROR,204,41.2,pos 1,1,0
1,Bloodseeker,ARTERIAL SPRAY,96,44.8,pos 1,4,0
2,Drow Ranger,VANTAGE POINT,177,45.8,pos 1,6,0
3,Juggernaut,BLADEFORM,273,48.7,pos 1,8,1
4,Shadow Fiend,LASTING PRESENCE,155,51.6,pos 1,11,0
...,...,...,...,...,...,...,...
85,Dark Willow,THROWING SHADE,123,43.9,pos 5,119,0
86,Grimstroke,INKSTIGATE,325,47.7,pos 5,121,0
87,Snapfire,RICOCHET II,916,47.5,pos 5,128,0
88,Marci,SIDEKICK,742,45.7,pos 5,136,0


In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
def make_config_wr(get_df_func, df, name, others):
    margin = 20
    width = 585 - margin
    height = 189 - margin

    return {
        "config_name": name,
        "categories": [
            {
                "category_name": "Carry",
                "x_position": 0.000000,
                "y_position": 0.000000,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 1),
            },
            {
                "category_name": "Mid",
                "x_position": 0.000000,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 2),
            },
            {
                "category_name": "Offlane",
                "x_position": 0.000000,
                "y_position": (height + margin) * 2,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 3),
            },
            {
                "category_name": "Semi-Support",
                "x_position": width + margin,
                "y_position": 0.000000,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 4),
            },
            {
                "category_name": "Full-Support",
                "x_position": width + margin,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 5),
            },
            {
                "category_name": "WR<50%",
                "x_position": width + margin,
                "y_position": (height + margin) * 2,
                "width": width / 3,
                "height": height,
                "hero_ids": others[0],
            },
            {
                "category_name": "WR<48%",
                "x_position": width + margin + width / 3,
                "y_position": (height + margin) * 2,
                "width": width / 3,
                "height": height,
                "hero_ids": others[1],
            },
            {
                "category_name": "WR<45%",
                "x_position": width + margin + 2 * width / 3,
                "y_position": (height + margin) * 2,
                "width": width / 3,
                "height": height,
                "hero_ids": others[2],
            },
        ],
    }

In [ ]:
def make_config(get_df_func, df, name):
    margin = 20
    width = 585 - margin
    height = 189 - margin
    not_included = [
        id
        for id in hero_ids
        if id
        not in [
            j for sub in [get_df_func(df, i) for i in [1, 2, 3, 4, 5]] for j in sub
        ]
    ]

    return {
        "config_name": name,
        "categories": [
            {
                "category_name": "Carry",
                "x_position": 0.000000,
                "y_position": 0.000000,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 1),
            },
            {
                "category_name": "Mid",
                "x_position": 0.000000,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 2),
            },
            {
                "category_name": "Offlane",
                "x_position": 0.000000,
                "y_position": (height + margin) * 2,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 3),
            },
            {
                "category_name": "Semi-Support",
                "x_position": width + margin,
                "y_position": 0.000000,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 4),
            },
            {
                "category_name": "Full-Support",
                "x_position": width + margin,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, 5),
            },
            {
                "category_name": "Not-Included",
                "x_position": width + margin,
                "y_position": (height + margin) * 2,
                "width": width,
                "height": height,
                "hero_ids": not_included,
            },
        ],
    }

In [ ]:
df_full_facets.sort_values("hero")

,hero,facet,matches,wr,pos,hero_id,facet_id
33,Abaddon,THE QUICKENING,211,55.9,pos 5,102,0
78,Abaddon,MEPHITIC SHROUD,355,52.1,pos 5,102,1
67,Alchemist,SEED MONEY,228,49.1,pos 4,73,0
22,Alchemist,SEED MONEY,96,39.6,pos 2,73,0
61,Alchemist,SEED MONEY,635,50.2,pos 1,73,0
...,...,...,...,...,...,...,...
7,Zeus,LIVEWIRE,263,54.0,pos 4,22,0
7,Zeus,LIVEWIRE,138,51.4,pos 5,22,0
52,Zeus,DIVINE RAMPAGE,66,56.1,pos 5,22,1
51,Zeus,DIVINE RAMPAGE,172,54.7,pos 4,22,1


In [ ]:
df_full_facets[
    (df_full_facets["pos"] == f"pos {5}") & (df_full_facets["matches"] >= 50) & (df_full_facets["wr"] > 51)
].sort_values(["wr"], ascending=False)

,hero,facet,matches,wr,pos,hero_id,facet_id
38,Oracle,CLAIRVOYANT CURSE,127,56.7,pos 5,111,0
52,Zeus,DIVINE RAMPAGE,66,56.1,pos 5,22,1
33,Abaddon,THE QUICKENING,211,55.9,pos 5,102,0
34,Elder Titan,DECONSTRUCTION,183,55.7,pos 5,103,0
71,Treant Protector,PRIMEVAL POWER,122,55.7,pos 5,83,0
...,...,...,...,...,...,...,...
1,Crystal Maiden,FROZEN EXPANSE,1520,51.8,pos 5,5,0
5,Tiny,CRASH LANDING,318,51.6,pos 5,19,0
28,Ogre Magi,LEARNING CURVE,735,51.6,pos 5,84,1
7,Zeus,LIVEWIRE,138,51.4,pos 5,22,0


In [ ]:
def make_config_facet(df, name, settings_num=1):
    max_height = 570
    max_width = 1180 - 100
    margin = 20
    width = max_width / 5 - margin
    height = max_height / 2 - margin
    
    height_3 = (max_height / 5) - 20
    width_3 = 70
    
    arr_df = []
    for position in [1,2,3,4,5]:
        if settings_num == 1:
            arr_df.append(df[
                (df["pos"] == f"pos {position}") & (df["matches"] > 200) & (df["wr"] > 51)
            ].sort_values(["wr"], ascending=False))
        if settings_num == 2:
            arr_df.append(df[
                (df["pos"] == f"pos {position}") & (df["matches"] > 50) & (df["wr"] > 51)
            ].sort_values(["wr"], ascending=False))
        
    return {
        "config_name": name,
        "categories": [
            {
                "category_name": "Carry Facet 1",
                "x_position": (width + margin) * 0,
                "y_position": (height + margin) * 0,
                "width": width,
                "height": height,
                "hero_ids": arr_df[0][arr_df[0]["facet_id"] == 0]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Mid Facet 1",
                "x_position": (width + margin) * 1,
                "y_position": (height + margin) * 0,
                "width": width,
                "height": height,
                "hero_ids": arr_df[1][arr_df[1]["facet_id"] == 0]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Offlane Facet 1",
                "x_position": (width + margin) * 2,
                "y_position": (height + margin) * 0,
                "width": width,
                "height": height,
                "hero_ids": arr_df[2][arr_df[2]["facet_id"] == 0]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Semi-Support Facet 1",
                "x_position": (width + margin) * 3,
                "y_position": (height + margin) * 0,
                "width": width,
                "height": height,
                "hero_ids": arr_df[3][arr_df[3]["facet_id"] == 0]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Full-Support Facet 1",
                "x_position": (width + margin) * 4,
                "y_position": (height + margin) * 0,
                "width": width,
                "height": height,
                "hero_ids": arr_df[4][arr_df[4]["facet_id"] == 0]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Carry Facet 2",
                "x_position": (width + margin) * 0,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": arr_df[0][arr_df[0]["facet_id"] == 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Mid Facet 2",
                "x_position": (width + margin) * 1,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": arr_df[1][arr_df[1]["facet_id"] == 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Offlane Facet 2",
                "x_position": (width + margin) * 2,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": arr_df[2][arr_df[2]["facet_id"] == 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Semi-Support Facet 2",
                "x_position": (width + margin) * 3,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": arr_df[3][arr_df[3]["facet_id"] == 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "Full-Support Facet 2",
                "x_position": (width + margin) * 4,
                "y_position": (height + margin) * 1,
                "width": width,
                "height": height,
                "hero_ids": arr_df[4][arr_df[4]["facet_id"] == 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "pos1 F 3+",
                "x_position": max_width + margin,
                "y_position": 0.000000,
                "width": width_3,
                "height": height_3,
                "hero_ids": arr_df[0][arr_df[0]["facet_id"] > 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "pos2 F 3+",
                "x_position": max_width + margin,
                "y_position": (height_3 + margin) * 1,
                "width": width_3,
                "height": height_3,
                "hero_ids": arr_df[1][arr_df[1]["facet_id"] > 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "pos3 F 3+",
                "x_position": max_width + margin,
                "y_position": (height_3 + margin) * 2,
                "width": width_3,
                "height": height_3,
                "hero_ids": arr_df[2][arr_df[2]["facet_id"] > 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "pos4 F 3+",
                "x_position": max_width + margin,
                "y_position": (height_3 + margin) * 3,
                "width": width_3,
                "height": height_3,
                "hero_ids": arr_df[3][arr_df[3]["facet_id"] > 1]["hero_id"].values.tolist(),
            },
            {
                "category_name": "pos5 F 3+",
                "x_position": max_width + margin,
                "y_position": (height_3 + margin) * 4,
                "width": width_3,
                "height": height_3,
                "hero_ids": arr_df[4][arr_df[4]["facet_id"] > 1]["hero_id"].values.tolist(),
            },
        ],
    }

In [ ]:
df = df_full
position = 5
(
    df[
        (df["pos"] == f"pos {position}") & (df["matches"] > 200) & (df["wr"] > 50)
    ].sort_values(["matches"], ascending=False)
)

,wr,pos,matches,contest,hero,rating,networth,phase1,phase2,phase3,expert,expertmatches,radiant,dire,elo,wr9500,hero_id
0,55.0,pos 5,2612,40.7,Sven,3605,2021,55.1,56.1,51.4,59.4,727,56.0,54.0,3605,54.9,18
3,52.5,pos 5,1856,76.7,Weaver,3292,2548,51.7,46.6,64.7,59.5,79,53.6,51.4,3292,51.5,63
2,52.2,pos 5,1758,31.1,Clockwerk,3314,2171,52.2,50.6,43.5,55.1,381,54.1,50.4,3314,52.9,51
7,51.7,pos 5,1701,14.0,Crystal Maiden,3217,2122,50.6,54.5,52.9,57.1,338,54.1,49.5,3217,50.2,5
5,53.2,pos 5,1658,11.7,Treant Protector,3240,2106,52.7,55.8,53.8,59.8,455,53.9,52.6,3240,52.3,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,55.8,pos 5,301,11.7,Batrider,3219,2457,54.1,56.1,60.0,57.7,52,60.3,51.9,3219,56.2,65
17,54.1,pos 5,290,42.9,Monkey King,3060,2510,57.6,51.4,44.4,0.0,0,54.1,54.2,3060,52.9,114
14,51.8,pos 5,253,6.9,Dazzle,3083,2443,53.6,43.5,45.5,45.8,48,49.6,53.9,3083,40.0,50
13,55.2,pos 5,248,10.4,Alchemist,3084,3069,56.8,49.2,58.3,62.1,58,61.8,48.8,3084,64.8,73


In [ ]:
def filer_heroes(df, pos=1, matches=0, wr=0, sort_by="wr"):
    return df[
        (df["pos"] == f"pos {pos}") & (df["matches"] > matches) & (df["wr"] > wr)
    ].sort_values([sort_by], ascending=False)
df = df_full
filer_heroes(df, 5, 200, 0, "matches")

,wr,pos,matches,contest,hero,rating,networth,phase1,phase2,phase3,expert,expertmatches,radiant,dire,elo,wr9500,hero_id
0,55.0,pos 5,2612,40.7,Sven,3605,2021,55.1,56.1,51.4,59.4,727,56.0,54.0,3605,54.9,18
49,50.0,pos 5,2534,27.9,Jakiro,2566,2212,50.6,46.4,56.1,56.1,417,50.8,49.2,2566,49.1,64
18,50.0,pos 5,2428,45.5,Phoenix,3055,2473,49.9,45.9,46.9,53.7,616,51.3,48.8,3055,49.2,110
62,46.4,pos 5,1889,68.8,Pudge,403,2343,47.7,42.7,36.7,57.3,96,49.8,42.4,403,42.7,14
3,52.5,pos 5,1856,76.7,Weaver,3292,2548,51.7,46.6,64.7,59.5,79,53.6,51.4,3292,51.5,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,48.8,pos 5,285,7.0,Silencer,2885,2113,50.4,46.9,44.0,56.5,23,49.2,48.4,2885,37.8,75
31,49.4,pos 5,259,11.9,Bounty Hunter,2894,2360,47.4,47.8,66.7,58.3,36,55.6,42.9,2894,49.5,62
14,51.8,pos 5,253,6.9,Dazzle,3083,2443,53.6,43.5,45.5,45.8,48,49.6,53.9,3083,40.0,50
13,55.2,pos 5,248,10.4,Alchemist,3084,3069,56.8,49.2,58.3,62.1,58,61.8,48.8,3084,64.8,73


In [ ]:
df = df_full
position = 5
df[
    (df["pos"] == f"pos {position}") 
    & (df["matches"] > 200) 
    & (df["wr"] > 50)
].sort_values(["wr"], ascending=False)

,wr,pos,matches,contest,hero,rating,networth,phase1,phase2,phase3,expert,expertmatches,radiant,dire,elo,wr9500,hero_id
6,55.8,pos 5,301,11.7,Batrider,3219,2457,54.1,56.1,60.0,57.7,52,60.3,51.9,3219,56.2,65
13,55.2,pos 5,248,10.4,Alchemist,3084,3069,56.8,49.2,58.3,62.1,58,61.8,48.8,3084,64.8,73
0,55.0,pos 5,2612,40.7,Sven,3605,2021,55.1,56.1,51.4,59.4,727,56.0,54.0,3605,54.9,18
4,54.9,pos 5,869,9.9,Oracle,3269,2164,53.2,59.9,47.8,56.9,167,53.7,56.2,3269,62.0,111
8,54.6,pos 5,438,7.4,Bane,3207,2133,57.4,44.6,52.9,57.7,104,59.2,50.2,3207,54.1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,51.7,pos 5,1701,14.0,Crystal Maiden,3217,2122,50.6,54.5,52.9,57.1,338,54.1,49.5,3217,50.2,5
15,51.1,pos 5,829,12.4,Ogre Magi,3068,2137,52.2,45.3,35.0,53.1,196,52.4,49.9,3068,49.8,84
12,50.9,pos 5,713,14.5,Io,3086,2306,51.2,54.3,30.8,59.4,187,51.1,50.7,3086,42.4,91
33,50.3,pos 5,618,44.1,Spirit Breaker,2867,2293,52.3,43.0,55.6,64.9,37,47.6,52.7,2867,49.3,71


In [ ]:
def get_by_rating(df: pd.DataFrame, position: int):
    return (
        df[(df["pos"] == f"pos {position}") & (df["matches"] > 200)]
        .sort_values(["rating"], ascending=False)["hero_id"]
        .values
        .tolist()
    )

In [ ]:
def get_expert(df: pd.DataFrame, position: int):
    return (
        df[
            (df["pos"] == f"pos {position}")
            & (df["expertmatches"] > 20)
            & (df["expert"] > 55)
            & (df["matches"] > 100)
        ]
        .sort_values(["expert"], ascending=False)["hero_id"]
        .values.tolist()
    )

In [ ]:
def get_popular(df: pd.DataFrame, position: int):
    return (
        df[(df["pos"] == f"pos {position}") & (df["matches"] > 200)]
        .sort_values(["matches"], ascending=False)["hero_id"]
        .values
        .tolist()
    )

In [ ]:
df_temp = df_full.groupby("hero").max()
df_temp[df_temp.wr < 50]

,wr,pos,matches,contest,rating,networth,phase1,phase2,phase3,expert,expertmatches,radiant,dire,elo,wr9500,hero_id
hero,,,,,,,,,,,,,,,,
Bloodseeker,48.9,pos 5,436,6.8,2932,4471,50.0,50.0,100.0,55.1,98,55.1,44.4,2932,100.0,4
Disruptor,49.4,pos 5,836,10.3,2844,2008,51.6,43.9,55.2,56.6,166,53.2,45.8,2844,50.6,87
Drow Ranger,46.0,pos 5,211,5.4,2731,4033,75.0,47.9,50.0,41.2,17,49.1,50.0,2731,44.3,6
Enchantress,46.7,pos 5,673,8.2,2885,4823,50.6,46.1,100.0,47.0,83,50.0,50.0,2885,66.7,58
Gyrocopter,46.4,pos 5,1499,13.0,2733,4601,100.0,48.6,80.0,66.7,106,62.5,50.0,2733,62.5,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Techies,48.8,pos 5,398,7.8,2927,4028,51.1,49.4,66.7,61.1,79,49.3,51.9,2927,50.9,105
Templar Assassin,49.5,pos 5,1495,27.4,2690,4693,37.3,50.0,66.7,49.2,132,49.7,49.2,2690,51.9,46
Venomancer,48.2,pos 5,1181,19.8,2855,4061,47.3,75.0,43.8,59.1,140,50.0,46.9,2855,50.0,40


In [ ]:
def get_wr(df: pd.DataFrame, position: int):
    return (
        df[(df["pos"] == f"pos {position}") & (df["matches"] > 200) & (df["wr"] > 50)]
        .sort_values(["wr"], ascending=False)["hero_id"]
        .values.tolist()
    )


def get_others_wr(df: pd.DataFrame):
    df_temp = df.groupby("hero").max()
    return [
        df_temp[(df_temp["matches"] > 100) & (df_temp["wr"] < 50) & (df_temp["wr"] > 48)]
        .sort_values(["wr"], ascending=False)["hero_id"]
        .values.tolist(),
        
        df_temp[(df_temp["matches"] > 100) & (df_temp["wr"] < 48) & (df_temp["wr"] > 45)]
        .sort_values(["wr"], ascending=False)["hero_id"]
        .values.tolist(),
        
        df_temp[(df_temp["matches"] > 100) & (df_temp["wr"] < 45)]
        .sort_values(["wr"], ascending=False)["hero_id"]
        .values.tolist(),
    ]

In [ ]:
config = {
    "version": 3,
    "configs": [
        make_config_facet(df_full_facets, "Facets matches>200"),
        make_config_facet(df_full_facets, "Facets All", 2),
        make_config_wr(get_wr, df_full, "Win rate", get_others_wr(df_full)),
        make_config(get_popular, df_full, "Popularity"),
        make_config(get_by_rating, df_full, "D2PT Rating"),
    ],
}

In [ ]:
driver.quit()

## Find Steam

In [ ]:
import os
import winreg
import win32api

def read_reg(ep, p = r"", k = ''):
    try:
        key = winreg.OpenKeyEx(ep, p)
        value = winreg.QueryValueEx(key,k)
        if key:
            winreg.CloseKey(key)
        return value[0]
    except Exception as e:
        return None
    return None

Path1 = "{}\\Microsoft\\Windows\\Start Menu\\Programs\\Steam\\Steam.lnk".format(os.getenv('APPDATA'))
if os.path.exists(Path1):
    import sys
    import win32com.client 

    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(Path1)
    Path1Res = shortcut.Targetpath
else:
    Path1Res = False
Path2 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Wow6432Node\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
Path3 = str(read_reg(ep = winreg.HKEY_LOCAL_MACHINE, p = r"SOFTWARE\Valve\Steam", k = 'InstallPath'))+r"\steam.exe"
if not os.path.exists(Path2):
    Path2 = None
if not os.path.exists(Path3):
    Path3 = None
PossiblePaths = [r"X:\Steam\steam.exe", r"X:\Program Files\Steam\steam.exe", r"X:\Program Files (x86)\Steam\steam.exe"]
ValidHardPaths = []
for Drive in win32api.GetLogicalDriveStrings().split('\000')[:-1]:
    Drive = Drive.replace(':\\', '')
    for path in PossiblePaths:
        path = path.replace("X", Drive)
        if os.path.exists(path):
            ValidHardPaths.append(path)
if len(ValidHardPaths) == 0:
    ValidHardPaths = ["None"]
ValidHardPaths

['C:\\Program Files (x86)\\Steam\\steam.exe']

In [ ]:
steam_dir = os.path.dirname(os.path.realpath(ValidHardPaths[0]))
steam_dir = os.path.join(steam_dir, "userdata")
steam_dir

'C:\\Program Files (x86)\\Steam\\userdata'

In [ ]:
dirs_to_check = [os.path.join(steam_dir, id, "570", "remote", "cfg") for id in os.listdir(steam_dir)]
cfg_dirs = [dir for dir in dirs_to_check if os.path.isdir(dir)]
cfg_dirs

['C:\\Program Files (x86)\\Steam\\userdata\\106615943\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\106675276\\570\\remote\\cfg',
 'C:\\Program Files (x86)\\Steam\\userdata\\383613970\\570\\remote\\cfg']

In [ ]:
for dir in cfg_dirs:
    with open(os.path.join(dir, "hero_grid_config.json"), 'w+') as f:
        json.dump(dict(config), f)

## Autorun

In [ ]:
import getpass
import os
import sys
USER_NAME = getpass.getuser()

In [ ]:
from win32com.client import Dispatch

bat_path = rf'C:\Users\{USER_NAME}\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Startup\{os.path.basename(sys.executable)}.lnk'
target = sys.executable

shell = Dispatch('WScript.Shell')
shortcut = shell.CreateShortCut(bat_path)
shortcut.Targetpath = target
shortcut.save()

In [ ]:
print("Done!")

for remaining in range(2, 0, -1):
    time.sleep(1)

Done!
